## Uniqlo Sales Webcrawler
https://www.selenium.dev/selenium/docs/api/py/api.html

In [100]:
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
import re
import pandas as pd
import numpy as np
pd.options.display.max_colwidth = 100

In [97]:
url = 'https://www.uniqlo.com/us/en/feature/sale/men'

options = Options()
# options.add_argument('--headless') 
driver = webdriver.Chrome(options=options)
driver.maximize_window()
driver.get(url)

# Load all products on the page
while True:
    try:
        time.sleep(3)
        driver.execute_script("window.scrollBy(0,document.body.scrollHeight)") # scroll to bottom 
        more_button = driver.find_element(By.XPATH, '//*[@id="root"]/div[9]/div/section[1]/div/div/section/div/div[3]/div/div[2]/button/span')
        more_button.click()
    except:
        print('Button not found')
        break

tiles = driver.find_elements(By.CSS_SELECTOR, "a[class*='uq-ec-product-tile']:not([data-category*='L3_sale_men'])")

df = pd.DataFrame(columns=['Items', 'Price', 'Sizes', 'Link'])

for i in range(0, len(tiles)):
    item = tiles[i].find_element(By.CSS_SELECTOR, "h3[class*='uq-ec-product']:not([style*='-webkit-line'])")

    price = tiles[i].find_element(By.CSS_SELECTOR, "p[class*='uq-ec-price-text']")
    
    size = tiles[i].find_element(By.CSS_SELECTOR, "p[class*='color-secondary']:not([class*='footer']")
    
    link = tiles[i].get_attribute('href')
    
    df.loc[i] = [item.text, price.text, size.text, link]
    
driver.quit()

Button not found


In [98]:
df['Link'] = df['Link'].str[:-2] + 'store-inventory' # extra two integers on end of link

In [154]:
print(df['Sizes'].to_string())

0             Unisex, XS-XXL
1                Men, XXS-XL
2                Unisex, XXL
3               Men, XXS-XXL
4                   Men, XXL
5                Men, XXS-XS
6                     Men, L
7               Unisex, XS-S
8            Unisex, XXS-3XL
9            Unisex, XXS-3XL
10           Unisex, XXS-3XL
11             Unisex, L-3XL
12           Unisex, XXS-3XL
13               Unisex, XXS
14               Unisex, XXS
15               Unisex, XXS
16             Unisex, XXS-M
17           Unisex, XXS-3XL
18               Unisex, XXS
19                Unisex, XL
20               Unisex, 3XL
21               Unisex, XXS
22            Unisex, XXS-XS
23                Unisex, XS
24           Unisex, XXS-XXL
25           Unisex, XXS-3XL
26           Unisex, XXS-3XL
27           Unisex, XXS-3XL
28           Unisex, XXS-3XL
29           Unisex, XXS-3XL
30             Unisex, S-3XL
31           Unisex, XXS-3XL
32           Unisex, XXS-3XL
33               Unisex, XXS
34           U

In [101]:
df['Availability'] = np.nan
df

,Items,Price,Sizes,Link,Availability
0,Indigo Dye Crew Neck Short-Sleeve T-Shirt,$14.90,"Unisex, XS-XXL",https://www.uniqlo.com/us/en/products/E446907-000/store-inventory,NaN
1,Relaxed Fit V-Neck Short-Sleeve T-Shirt (Theory),$14.90,"Men, XXS-XL",https://www.uniqlo.com/us/en/products/E445168-000/store-inventory,NaN
2,Supima® Cotton Crew Neck Short-Sleeve T-Shirt,$5.90,"Unisex, XXL",https://www.uniqlo.com/us/en/products/E422990-000/store-inventory,NaN
3,Crew Neck Short-Sleeve T-Shirt (MARNI),$19.90,"Men, XXS-XXL",https://www.uniqlo.com/us/en/products/E451483-000/store-inventory,NaN
4,Striped Crew Neck Short-Sleeve T-Shirt (MARNI),$19.90,"Men, XXL",https://www.uniqlo.com/us/en/products/E449634-000/store-inventory,NaN
...,...,...,...,...,...
119,Faux Suede Lace-Up Sneakers,$29.90,"Unisex, 4.5-7.0",https://www.uniqlo.com/us/en/products/E442233-000/store-inventory,NaN
120,Cotton Canvas Lace Up Sneakers,$29.90,"Unisex, 4.5-9.5",https://www.uniqlo.com/us/en/products/E434989-000/store-inventory,NaN
121,Small Plate (The Louvre),$5.90,Unisex,https://www.uniqlo.com/us/en/products/E447852-000/store-inventory,NaN
122,Small Plate (The Louvre),$5.90,Unisex,https://www.uniqlo.com/us/en/products/E447851-000/store-inventory,NaN


In [158]:
size = 'XS'
zip_code = '08854'
store = 'UNIQLO Menlo Park '

for i in range(73, len(df['Link'])):
    options = Options()
    options.add_argument('--headless') 
    driver = webdriver.Chrome(options=options)
    driver.maximize_window()
    
    driver.get(df['Link'][i])

    driver.execute_script('window.scrollBy(0,300)')
    
    try:
        size_btn = WebDriverWait(driver, 3).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, f"input[aria-label={size}]"))
                )
        size_btn.click()
    except:
        continue
    
#     size_btn = driver.find_element(By.CSS_SELECTOR, f"input[aria-label={size}]")
#     size_btn.click()

    zip_bar = driver.find_element(By.ID, 'o2o-store-search')
    zip_bar.send_keys(zip_code) # search for "test"

    search_btn = driver.find_element(By.XPATH, "//button[text()='Search']")

    colors = driver.find_element(By.ID, 'product-color-picker').find_elements(By.CSS_SELECTOR, "div[class*='uq-ec-chip--short']")

    color_avail = {}

    for color in colors:
        color.click()
        
        try:
            search_btn.click()
        except:
            continue
        
        color_txt = driver.find_element(By.XPATH, "//label[text()='Color']/following-sibling::label").text

        time.sleep(1)
        driver.execute_script('window.scrollBy(0,500)')
        
        avail = WebDriverWait(driver, 5).until(
                    EC.presence_of_element_located((By.XPATH, f"//span[text()='{store}']/ancestor::ul/following-sibling::div/span/span"))
                )
#         print(avail.text)

        driver.execute_script('window.scrollBy(0,-500)')

        color_avail[color_txt] = avail.text

    print(color_avail)
    df['Availability'][i] = color_avail
    
driver.quit()

{'02 LIGHT GRAY': 'Out of stock', '09 BLACK': 'Out of stock'}
{'06 GRAY': 'Out of stock', '09 BLACK': 'Out of stock', '69 NAVY': 'Out of stock'}
{'09 BLACK': 'Out of stock', '32 BEIGE': 'Out of stock', '34 BROWN': 'Out of stock', '57 OLIVE': 'Out of stock', '69 NAVY': 'Out of stock'}
{'00 WHITE': 'Out of stock', '05 GRAY': 'Out of stock', '09 BLACK': 'Out of stock', '32 BEIGE': 'Out of stock', '57 OLIVE': 'Out of stock', '64 BLUE': 'Out of stock', '69 NAVY': 'Out of stock'}
{'03 GRAY': 'Out of stock', '09 BLACK': 'Out of stock', '58 DARK GREEN': 'Out of stock', '69 NAVY': 'Out of stock'}
{'08 DARK GRAY': 'Out of stock', '12 PINK': 'Out of stock', '27 ORANGE': 'Out of stock', '35 BROWN': 'Out of stock'}
{'05 GRAY': 'Out of stock', '09 BLACK': 'Out of stock', '30 NATURAL': 'Out of stock', '58 DARK GREEN': 'Out of stock', '68 BLUE': 'Out of stock', '69 NAVY': 'Out of stock'}
{'63 BLUE': 'Out of stock'}
{'01 OFF WHITE': 'Out of stock', '68 BLUE': 'Out of stock'}
{'32 BEIGE': 'Out of stock'

In [159]:
print(df.to_string())

                                                                             Items    Price                  Sizes                                                               Link                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               Availability
0                                        Indigo Dye Crew Neck Short-Sleeve T-Shirt   $14.90         Unisex, XS-XXL  https://www.uniqlo.com/us/en/products/E446907-000/store-inventory                                                                  